In [1]:
# %%

import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import json
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import random

random.seed(42)

In [2]:
# %%

import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import json
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import random

random.seed(42)

In [3]:
# %%

# Load the data from the JSON file
with open('data/gushiwen.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(len(data))

author_count = defaultdict(int)

# Count the occurrences of each author
for item in data:
    author_count[item['author']] += 1

# Sort authors by the number of occurrences
sorted_authors = sorted(author_count.items(), key=lambda x: x[1], reverse=True)
top_twenty_authors = [item[0] for item in sorted_authors[1:31]]
print(top_twenty_authors)

FileNotFoundError: [Errno 2] No such file or directory: 'data/gushiwen.json'

In [ ]:
# %%

filtered = list(filter(lambda x: x['author'] in top_twenty_authors, data))
filtered = list(filter(lambda x: '(' not in x['content'] and ')' not in x['content'], filtered))
contents = [item['content'].replace("<br/>", "").strip() for item in filtered]
contents = list(filter(lambda x: len(x) in [24, 32, 48] and all(c in '，。' or '\u4e00' <= c <= '\u9fa5' for c in x), contents))
print(len(contents))
contents[:10]

# 统计古诗格式分布
poem_formats = {
    "五言绝句": 0,  # 20个字左右（4行，每行5字）
    "七言绝句": 0,  # 28个字左右（4行，每行7字）
    "五言律诗": 0,  # 40个字左右（8行，每行5字）
    "七言律诗": 0,  # 56个字左右（8行，每行7字）
    "其他": 0
}
# 创建字典来存储每种格式的诗歌样本
poem_samples = {
    "五言绝句": [],
    "七言绝句": [],
    "五言律诗": [],
    "七言律诗": [],
    "其他": []
}

for content in contents:
    length = len(content)
    # 考虑标点符号，实际字数会少一些
    if length == 24:  # 五言绝句（考虑可能的变体）
        poem_formats["五言绝句"] += 1
        poem_samples["五言绝句"].append(content)
    elif length == 32:  # 七言绝句
        poem_formats["七言绝句"] += 1
        poem_samples["七言绝句"].append(content)
    elif length == 48:  # 五言律诗
        poem_formats["五言律诗"] += 1
        poem_samples["五言律诗"].append(content)
    elif length == 64:  # 七言律诗
        poem_formats["七言律诗"] += 1
        poem_samples["七言律诗"].append(content)
    else:
        poem_formats["其他"] += 1
        poem_samples["其他"].append(content)

# 打印每种格式的两个样本
print("\n各格式诗歌样本:")
for format_name, samples in poem_samples.items():
    print(f"\n{format_name}样本:")
    for i, sample in enumerate(samples[:4]):
        print(f"  样本{i+1}: {sample}")

total = len(contents)
percentages = {format_name: count/total*100 for format_name, count in poem_formats.items()}

print("\n古诗格式分布:")
for format_name, count in poem_formats.items():
    print(f"{format_name}: {count} 首 ({percentages[format_name]:.2f}%)")

8047

各格式诗歌样本:

五言绝句样本:
  样本1: 床前明月光，疑是地上霜。举头望明月，低头思故乡。
  样本2: 危楼高百尺，手可摘星辰。不敢高声语，恐惊天上人。
  样本3: 懒摇白羽扇，裸袒青林中。脱巾挂石壁，露顶洒松风。
  样本4: 小时不识月，呼作白玉盘。又疑瑶台镜，飞在青云端。

七言绝句样本:
  样本1: 朝辞白帝彩云间，千里江陵一日还。两岸猿声啼不住，轻舟已过万重山。
  样本2: 天门中断楚江开，碧水东流至此回。两岸青山相对出，孤帆一片日边来。
  样本3: 日照香炉生紫烟，遥看瀑布挂前川。飞流直下三千尺，疑是银河落九天。
  样本4: 李白乘舟将欲行，忽闻岸上踏歌声。桃花潭水深千尺，不及汪伦送我情。

五言律诗样本:
  样本1: 渡远荆门外，来从楚国游。山随平野尽，江入大荒流。月下飞天镜，云生结海楼。仍怜故乡水，万里送行舟。
  样本2: 青山横北郭，白水绕东城。此地一为别，孤蓬万里征。浮云游子意，落日故人情。挥手自兹去，萧萧班马鸣。
  样本3: 五月天山雪，无花只有寒。笛中闻折柳，春色未曾看。晓战随金鼓，宵眠抱玉鞍。愿将腰下剑，直为斩楼兰。
  样本4: 群峭碧摩天，逍遥不记年。拨云寻古道，倚石听流泉。花暖青牛卧，松高白鹤眠。语来江色暮，独自下寒烟。

七言律诗样本:

其他样本:

古诗格式分布:
五言绝句: 734 首 (9.12%)
七言绝句: 2763 首 (34.34%)
五言律诗: 4550 首 (56.54%)
七言律诗: 0 首 (0.00%)
其他: 0 首 (0.00%)


In [ ]:
# %%

char_freq = defaultdict(int)
min_freq = 3
# Count the frequency of each character in contents
for content in contents:
    for char in content:
        char_freq[char] += 1
vocab = ['<pad>', '<sos>', '<eos>', '<unk>', '<五绝>', '<七绝>', '<五律>']

unk_count = 0
for char, freq in char_freq.items():
    if freq >= min_freq:
        vocab.append(char)
    else:
        unk_count += freq

total_count = sum(char_freq.values())
unk_ratio = unk_count / total_count if total_count > 0 else 0
print(f"unk ratio: {unk_count}/{total_count} = {unk_ratio * 100:.2f}%")

vocab_size = len(vocab)
print("vocab_size:", vocab_size)
max_length = max(len(content) for content in contents) + 3
print("max_length:", max_length)
stoi = {c: i for i, c in enumerate(vocab)}
itos = {i: c for c, i in stoi.items()}

unk ratio: 1822/324432 = 0.56%
vocab_size: 3450
max_length: 51


In [ ]:
# %%

def get_format_token(content):
    if len(content) == 24:
        return '<五绝>'
    elif len(content) == 32:
        return '<七绝>'
    elif len(content) == 48:
        return '<五律>'


def encode(seq, add_sos=True, add_eos=True, format_token=None):
    indices = [stoi[c] if c in stoi else stoi['<unk>'] for c in seq]
    if add_sos:
        indices = [stoi['<sos>']] + indices
    if format_token:
        indices = [stoi[format_token]] + indices
    if add_eos:
        indices = indices + [stoi['<eos>']]
    return indices


def decode(indices, align=False):
    def decode_token(t):
        if isinstance(t, int) and t != -1:
            return itos[t].replace('<unk>', '？')
        elif isinstance(t, torch.Tensor) and t.item() != -1:
            return itos[t.item()].replace('<unk>', '？')
        else:
            return 'X'
    result = [decode_token(idx) for idx in indices]
    if align:
        return "|".join([f"{c:^5}" for c in result])
    return "".join(result)


def build_dataset(contents):
    xs = torch.zeros((len(contents), max_length), dtype=torch.long)
    ys = torch.full((len(contents), max_length), dtype=torch.long, fill_value=-1)
    for i, content in enumerate(contents):
        encoded = encode(content, format_token=get_format_token(content))
        xs[i, :len(encoded)] = torch.tensor(encoded)
        ys[i, 1:len(encoded)-1] = torch.tensor(encoded[2:])
    return xs, ys


# Randomly shuffle the contents
random.shuffle(contents)

# Split the contents into training and testing sets (80% train, 20% test)
split_index = int(len(contents) * 0.9)
train_contents = contents[:split_index]
test_contents = contents[split_index:]


train_xs, train_ys = build_dataset(train_contents)
test_xs, test_ys = build_dataset(test_contents)
print(decode(train_xs[0], align=True))
print(decode(train_ys[0], align=True))
train_set = TensorDataset(train_xs, train_ys)
test_set = TensorDataset(test_xs, test_ys)

<七绝> |<sos>|  将  |  军  |  作  |  镇  |  古  |  ？  |  洲  |  ，  |  水  |  腻  |  山  |  春  |  节  |  气  |  柔  |  。  |  清  |  夜  |  满  |  城  |  丝  |  管  |  散  |  ，  |  行  |  人  |  不  |  信  |  是  |  边  |  头  |  。  |<eos>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>|<pad>
  X  |  将  |  军  |  作  |  镇  |  古  |  ？  |  洲  |  ，  |  水  |  腻  |  山  |  春  |  节  |  气  |  柔  |  。  |  清  |  夜  |  满  |  城  |  丝  |  管  |  散  |  ，  |  行  |  人  |  不  |  信  |  是  |  边  |  头  |  。  |<eos>|  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  |  X  


In [ ]:
# %%

class EarlyStopping:
    def __init__(self, patience=5, verbose=False, min_delta=0):
        self.patience = patience
        self.verbose = verbose
        self.min_delta = min_delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.improvement_counter = 0

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score >= self.best_score - self.min_delta:
            self.improvement_counter = 0
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.improvement_counter += 1
            if self.improvement_counter >= 3:
                self.counter = 0  # 多次改善才清零

In [ ]:
# %%

class Head(nn.Module):
    def __init__(self, n_embd, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size)
        self.key = nn.Linear(n_embd, head_size)
        self.value = nn.Linear(n_embd, head_size)
        self.register_buffer("mask", torch.tril(torch.ones(2048, 2048)))  # 创建下三角掩码

    def forward(self, x):
        # x: (B, T, n_embd)
        B, T = x.shape[0], x.shape[1]
        q = self.query(x)  # (B, T, head_size)
        k = self.key(x)
        v = self.value(x)
        attn = q @ k.transpose(-2, -1)  # (B, T, T)
        attn = attn * (k.size(-1) ** -0.5)
        attn = attn.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)  # (B, T, T)

        out = attn @ v  # (B, T, head_size)
        return out


class MultiHead(nn.Module):
    def __init__(self, n_embd, head_size, n_head):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, head_size) for _ in range(n_head)])
        self.fc = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        attns = [head(x) for head in self.heads]
        out = torch.cat(attns, dim=-1)
        out = self.dropout(self.fc(out))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # 扩展维度
            nn.GELU(),  # 激活函数
            nn.Linear(4 * n_embd, n_embd),  # 恢复维度
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd, head_size, n_head, dropout=0.3):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHead(n_embd, head_size, n_head)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ff = FeedForward(n_embd, dropout)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x


class TransformerLM(nn.Module):
    def __init__(self, n_embd, n_head, n_layers, vocab_size, dropout=0.3):
        super().__init__()
        head_size = n_embd // n_head
        self.embedding = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding = nn.Embedding(2048, n_embd)
        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.Sequential(*[Block(n_embd, head_size, n_head, dropout) for _ in range(n_layers)])
        self.ln_in = nn.LayerNorm(n_embd)
        self.ln_f = nn.LayerNorm(n_embd)
        # self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, y=None):
        # idx: (B, T), y: (B, T)
        B, T = idx.shape
        tok_emb = self.embedding(idx)  # (B, T, n_embd)
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0).repeat(B, 1)  # (B, T)
        pos_emb = self.pos_embedding(pos)  # (B, T, n_embd)
        x = self.ln_in(tok_emb + pos_emb)
        x = self.dropout(x)
        x = self.blocks(x)
        x = self.ln_f(x)
        # logits = self.lm_head(x)  # (B, T, vocab_size)
        logits = F.linear(x, self.embedding.weight)  # (B, T, vocab_size)
        loss = None
        if y is not None:
            loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), y.view(-1), ignore_index=-1)
        return logits, loss

In [ ]:
# %%

n_embd = 128
batch_size = 32
n_head = 8
n_layers = 4
model = TransformerLM(n_embd, n_head, n_layers, vocab_size)
early_stopping = EarlyStopping(patience=5, verbose=True, min_delta=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=5e-2)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
# %%

def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    total_samples = 0
    with torch.no_grad():
        for x, y in data_loader:
            logits, loss = model(x, y)
            total_loss += loss.item() * x.size(0)
            total_samples += x.size(0)
    model.train()
    return total_loss / total_samples

In [ ]:
# %%

max_epoch = 300


def print_progress(epoch, i, train_loader, loss):
    progress = (i + 1) / len(train_loader) * 100
    bar_length = 30  # Length of the progress bar
    filled_length = int(bar_length * progress // 100)
    bar = '━' * filled_length + '─' * (bar_length - filled_length)
    print(f"epoch {epoch}, progress |{bar}| {progress:.0f}%, loss {loss.item()}", end='\r')


for epoch in range(max_epoch):
    i = 0
    epoch_losses = []
    for i, (x, y) in enumerate(train_loader):
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i > 0 and i % (len(train_loader) // 10) == 0:
            print_progress(epoch, i, train_loader, loss)
        epoch_losses.append(loss.item())
    avg_loss = sum(epoch_losses) / len(epoch_losses)
    test_loss = evaluate(model, test_loader)
    print(f"\r{' '* 100}", end="\r")  # 先清除当前行
    print(f"epoch {epoch}, train_avg_loss {avg_loss:.4f}, test_loss {test_loss:.4f}")
    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping at epoch", epoch, "with test loss", test_loss)
        break

epoch 0, train_avg_loss 14.1331, test_loss 6.7369                                                   
epoch 1, train_avg_loss 7.4386, test_loss 5.9915                                                    
epoch 2, train_avg_loss 6.4290, test_loss 5.8101                                                    
epoch 3, train_avg_loss 5.9807, test_loss 5.7547                                                    
epoch 4, train_avg_loss 5.8473, test_loss 5.7183                                                    
epoch 5, train_avg_loss 5.7844, test_loss 5.6944                                                    
epoch 6, train_avg_loss 5.7422, test_loss 5.6742                                                    
epoch 7, train_avg_loss 5.7058, test_loss 5.6409                                                    
epoch 8, train_avg_loss 5.6724, test_loss 5.6159                                                    
epoch 9, train_avg_loss 5.6383, test_loss 5.5822                                           

In [ ]:
# %%

@torch.no_grad()
def generate(model, max_new_tokens, context, temperature=1.0, top_k=20):
    model.eval()
    for _ in range(max_new_tokens):
        logits, _ = model(context)
        logits = logits[:, -1, :] / temperature
        # 只保留概率最高的top_k个选项
        v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
        logits[logits < v[:, [-1]]] = -float('Inf')
        probs = F.softmax(logits, dim=-1)
        input_next = torch.multinomial(probs, num_samples=1)
        # 检查是否所有批次都生成了结束标记
        if (input_next[:, 0] == stoi['<eos>']).all():
            break
        context = torch.cat((context, input_next), dim=-1)
    model.train()
    return context


def sample(text, count, model, max_new_tokens, temperature=1.0, format_token=None):
    context = torch.tensor(encode(text, add_sos=True, add_eos=False, format_token=format_token), dtype=torch.long)
    context = context.unsqueeze(0).repeat(count, 1)
    idx = generate(model, max_new_tokens, context, temperature)
    result = []
    for i in idx:
        eos_idx = (i == stoi['<eos>']).nonzero()
        if eos_idx.numel() > 0:
            i = i[:eos_idx[0]]
        result.append(decode(i))
    return result


for temperature in [0.6]:
    print(f"temperature: {temperature}")
    print("\n".join(sample("春眠", 20, model, 100, temperature=temperature, format_token='<五绝>')))

temperature: 0.6
<五绝><sos>春眠春水好，独倚洞房开。莫作山僧去，无人不见来。
<五绝><sos>春眠秋月夜，晚与一声中。自有人间事，不知不是风。
<五绝><sos>春眠千万仞，秋色一声清。应是长生事，不闻青琐行。
<五绝><sos>春眠花落尽，日暮雨凄凄。不是风尘泪，何人醉一壶。
<五绝><sos>春眠春欲尽，春色复凄凄。不见春风里，何人到此时。
<五绝><sos>春眠花下绿，春色夜初晴。只是南山路，今朝入郭城。
<五绝><sos>春眠一声声，夜坐一声清。何事秋山客，长歌不可惊。
<五绝><sos>春眠风景好，风露晚凄凄。莫道春犹落，空斋不可携。
<五绝><sos>春眠风景好，欲与雨声空。不是无人事，偏嫌一夜风。
<五绝><sos>春眠不可见，昨夜不成时。今夜更长夜，今宵却到时。
<五绝><sos>春眠千里别，暮雨一相随。更有东风雪，长吟一夜时。
<五绝><sos>春眠无限意，花落似花时。莫道人间事，无人莫是诗。
<五绝><sos>春眠一杯酒，晓色几人行。欲问长安在，还闻此处行。
<五绝><sos>春眠新雨后，夜坐一回环。何处风吹角，今宵不解携。
<五绝><sos>春眠秋气急，夜月夜同倾。谁是秋风里，人间夜宿行。
<五绝><sos>春眠红叶低，风雪绿苔垂。欲问三杯酒，时时一觉时。
<五绝><sos>春眠无限意，不解是人间。不是花前事，无妨得此闲。
<五绝><sos>春眠春不尽，独宿不同游。欲问南山客，还闻此路愁。
<五绝><sos>春眠花满地，秋色满江城。今日无人赏，时时是此行。
<五绝><sos>春眠不可见，日暮更纷纷。犹有山中月，何如水上云。
